# Lab 3.2.3: AWQ Quantization

**Module:** 3.2 - Model Quantization & Optimization  
**Time:** 1.5 hours  
**Difficulty:** ⭐⭐⭐⭐☆

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- [ ] Understand the AWQ (Activation-aware Weight Quantization) algorithm
- [ ] Compare AWQ to GPTQ on the same model
- [ ] Quantize a model using AutoAWQ
- [ ] Understand when to choose AWQ vs GPTQ

---

## 📚 Prerequisites

- Completed: Lab 3.2.2 (GPTQ Quantization)
- Knowledge of: GPTQ basics, quantization fundamentals
- Hardware: DGX Spark with 128GB unified memory

---

## 🌍 Real-World Context

**The Problem with GPTQ:** While GPTQ works great, it treats all weights equally. But some weights are more important than others!

**The AWQ Insight:** If we look at *activation magnitudes* during inference, we can identify which weights process the largest values. These "salient" weights should be quantized more carefully.

**Real-World Impact:**
| Metric | GPTQ | AWQ |
|--------|------|-----|
| Perplexity degradation | ~0.3-0.5 | ~0.2-0.4 |
| Reasoning tasks | Good | Better |
| Code generation | Good | Better |

---

## 🧒 ELI5: What is AWQ?

> **Imagine you're packing for a trip with limited suitcase space...**
>
> **GPTQ approach:** Shrink all your clothes equally with a compression bag.
> - Your formal suit gets as wrinkled as your casual t-shirts.
>
> **AWQ approach:** Before compressing, check which clothes you'll wear most often.
> - Keep your most-used items full-size
> - Compress the rarely-worn items more aggressively
> - Same suitcase space, but your important clothes stay nice!
>
> **In AI terms:** AWQ looks at which weights process the biggest activations (most important), and protects those weights from aggressive quantization. The result: same compression ratio, better quality!

---

## Part 1: Understanding AWQ

### The Key Insight: Weight Salience

Consider a weight matrix `W` and input activations `X`. The output is `Y = XW`.

If a particular weight column `w_i` typically multiplies with large activation values `x_i`, then:
- Any error in `w_i` gets **amplified** by `x_i`
- That weight is "salient" and should be protected

### AWQ's Solution: Per-Channel Scaling

Instead of quantizing `W` directly, AWQ:
1. Computes activation statistics to find salient channels
2. Applies per-channel scaling: `W' = W * diag(s)`, `X' = X * diag(1/s)`
3. Quantizes the scaled weights `W'`
4. The scaling protects salient weights from quantization error!

The math ensures: `X'W' = X * diag(1/s) * W * diag(s) = XW` ✓

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import math

print("="*60)
print("DGX Spark Environment Check")
print("="*60)

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory available: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Visualize the AWQ concept

def demonstrate_awq_concept():
    """
    Demonstrate how AWQ protects salient weights.
    """
    torch.manual_seed(42)
    
    # Create a weight matrix and activation matrix
    W = torch.randn(64, 64) * 0.5
    X = torch.randn(32, 64)
    
    # Some channels have much larger activations (salient)
    X[:, 10:15] *= 10  # Make channels 10-15 salient
    X[:, 50:55] *= 8   # Make channels 50-55 somewhat salient
    
    # Compute original output
    Y_original = X @ W
    
    # Method 1: Naive quantization (like basic INT4)
    def naive_quantize(tensor, bits=4):
        max_val = tensor.abs().max()
        scale = max_val / (2**(bits-1) - 1)
        quantized = torch.round(tensor / scale).clamp(-2**(bits-1), 2**(bits-1)-1)
        return quantized * scale
    
    W_naive = naive_quantize(W)
    Y_naive = X @ W_naive
    error_naive = (Y_original - Y_naive).pow(2).mean().sqrt()
    
    # Method 2: AWQ-style (protect salient channels)
    # Compute per-channel activation magnitudes
    activation_magnitude = X.abs().mean(dim=0)
    
    # Scale factor: protect channels with large activations
    # AWQ uses a more sophisticated formula, this is simplified
    s = (activation_magnitude / activation_magnitude.mean()).pow(0.5).clamp(min=0.1)
    
    # Scale weights and activations
    W_scaled = W * s.unsqueeze(0)  # Scale weight rows
    X_scaled = X / s.unsqueeze(0)  # Inverse scale activations
    
    # Quantize scaled weights
    W_awq = naive_quantize(W_scaled)
    Y_awq = X_scaled @ W_awq
    error_awq = (Y_original - Y_awq).pow(2).mean().sqrt()
    
    # Visualize
    fig, axes = plt.subplots(2, 3, figsize=(15, 8))
    
    # Row 1: Show the problem
    axes[0, 0].bar(range(64), activation_magnitude.numpy())
    axes[0, 0].set_title('Activation Magnitude by Channel')
    axes[0, 0].set_xlabel('Channel')
    axes[0, 0].axhline(y=activation_magnitude.mean(), color='r', linestyle='--', label='Mean')
    axes[0, 0].legend()
    
    axes[0, 1].bar(range(64), s.numpy())
    axes[0, 1].set_title('AWQ Scaling Factors')
    axes[0, 1].set_xlabel('Channel')
    axes[0, 1].axhline(y=1.0, color='r', linestyle='--', label='No scaling')
    axes[0, 1].legend()
    
    # Show weight distribution before/after scaling
    axes[0, 2].hist(W.flatten().numpy(), bins=50, alpha=0.5, label='Original')
    axes[0, 2].hist(W_scaled.flatten().numpy(), bins=50, alpha=0.5, label='Scaled')
    axes[0, 2].set_title('Weight Distribution')
    axes[0, 2].legend()
    
    # Row 2: Show the results
    axes[1, 0].imshow(Y_original.numpy()[:10], aspect='auto', cmap='RdBu')
    axes[1, 0].set_title('Original Output')
    
    axes[1, 1].imshow((Y_original - Y_naive).abs().numpy()[:10], aspect='auto', cmap='hot')
    axes[1, 1].set_title(f'Naive Quant Error (RMSE={error_naive:.4f})')
    
    axes[1, 2].imshow((Y_original - Y_awq).abs().numpy()[:10], aspect='auto', cmap='hot')
    axes[1, 2].set_title(f'AWQ Error (RMSE={error_awq:.4f})')
    
    plt.tight_layout()
    plt.savefig('awq_concept.png', dpi=150, bbox_inches='tight')
    plt.show()
    plt.close(fig)  # Free memory from figure
    
    print(f"\n📊 Error Comparison:")
    print(f"   Naive quantization RMSE: {error_naive:.4f}")
    print(f"   AWQ quantization RMSE:   {error_awq:.4f}")
    print(f"   Improvement: {(1 - error_awq/error_naive)*100:.1f}% lower error!")

demonstrate_awq_concept()

### 🔍 What Just Happened?

1. We created a scenario with **salient channels** (channels 10-15 and 50-55 have large activations)
2. **Naive quantization** introduces large errors because it ignores which weights matter most
3. **AWQ-style quantization** uses scaling to protect important weights, reducing error significantly

The key insight: **not all weights are equally important!**

---

## Part 2: Quantizing with AutoAWQ

Let's use the AutoAWQ library to quantize a real model and compare with GPTQ.

In [ ]:
# Install AutoAWQ if needed
# Note: On DGX Spark (ARM64), prefer using pre-installed NGC container packages

try:
    from awq import AutoAWQForCausalLM
    print("✅ AutoAWQ is available!")
except ImportError:
    print("Installing AutoAWQ...")
    print("⚠️  On DGX Spark (ARM64), this may compile CUDA kernels from source.")
    print("   This can take 5-10 minutes. Please be patient...")

    import subprocess
    result = subprocess.run(
        ["pip", "install", "autoawq", "--no-cache-dir"],
        capture_output=True,
        text=True
    )

    if result.returncode != 0:
        print(f"❌ Installation failed!")
        print("Error output (last 1000 chars):")
        print(result.stderr[-1000:] if len(result.stderr) > 1000 else result.stderr)
        print("\n💡 Solution: Use an NGC container with AutoAWQ pre-installed")
        raise ImportError("AutoAWQ installation failed - see error above")

    from awq import AutoAWQForCausalLM
    print("✅ AutoAWQ installed successfully!")

from transformers import AutoTokenizer, AutoModelForCausalLM
import gc

In [ ]:
# Use the same model as GPTQ for fair comparison
model_id = "facebook/opt-350m"

print(f"Model: {model_id}")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"Tokenizer loaded. Vocab size: {len(tokenizer)}")

In [ ]:
# AWQ Quantization Configuration
awq_config = {
    "zero_point": True,      # Use zero-point quantization
    "q_group_size": 128,     # Group size (same as GPTQ for comparison)
    "w_bit": 4,              # 4-bit quantization
    "version": "GEMM"        # Use GEMM kernel (faster)
}

print("AWQ Configuration:")
for k, v in awq_config.items():
    print(f"  {k}: {v}")

In [ ]:
# Prepare calibration data
# AWQ needs fewer samples than GPTQ (typically 128 is enough)

calibration_texts = [
    "The field of machine learning has grown exponentially in recent years.",
    "Artificial intelligence systems can now perform complex reasoning.",
    "Large language models are transforming how we interact with computers.",
    "Neural networks consist of interconnected layers of artificial neurons.",
    "Deep learning enables breakthroughs in computer vision and NLP.",
    "The transformer architecture revolutionized sequence modeling.",
    "Quantization reduces model size while maintaining performance.",
    "GPU acceleration enables training of billion-parameter models.",
    "Transfer learning allows models to leverage pre-trained knowledge.",
    "Attention mechanisms help models focus on relevant information.",
    "In 1969, humans first landed on the moon during Apollo 11.",
    "The capital of France is Paris, known for the Eiffel Tower.",
    "Python is a popular programming language for data science.",
    "Climate change poses significant challenges to ecosystems.",
    "Quantum computing may revolutionize cryptography.",
    "The stock market experienced volatility last quarter.",
    "Healthy eating contributes to overall well-being.",
    "Space exploration expands our understanding of the universe.",
    "Renewable energy sources are becoming cost-effective.",
    "The history of mathematics spans thousands of years.",
] * 8  # Repeat to get ~160 samples

print(f"Prepared {len(calibration_texts)} calibration samples")

In [ ]:
# Clear memory before quantization
gc.collect()
torch.cuda.empty_cache()

print(f"Initial GPU memory: {torch.cuda.memory_allocated()/1e9:.2f} GB")

In [ ]:
# Quantize with AWQ
print("="*60)
print("Quantizing with AWQ...")
print("="*60)

start_time = time.time()

# Load model for AWQ quantization
model_awq = AutoAWQForCausalLM.from_pretrained(
    model_id,
    device_map="cuda",
    safetensors=True
)

# Perform quantization
model_awq.quantize(
    tokenizer,
    quant_config=awq_config,
    calib_data=calibration_texts,
)

awq_time = time.time() - start_time
print(f"\nQuantization time: {awq_time:.1f} seconds")
print(f"GPU memory: {torch.cuda.memory_allocated()/1e9:.2f} GB")

In [ ]:
# Save the AWQ quantized model
import os

save_dir_awq = "./quantized_models/opt-350m-awq-4bit-g128"
os.makedirs(save_dir_awq, exist_ok=True)

print(f"Saving AWQ model to {save_dir_awq}...")
model_awq.save_quantized(save_dir_awq)
tokenizer.save_pretrained(save_dir_awq)

# Check file sizes
total_size = 0
print("\nSaved files:")
for f in os.listdir(save_dir_awq):
    size = os.path.getsize(os.path.join(save_dir_awq, f))
    total_size += size
    if size > 1e6:  # Only show large files
        print(f"  {f}: {size/1e6:.2f} MB")
print(f"\nTotal: {total_size/1e6:.2f} MB")

---

## Part 3: AWQ vs GPTQ Comparison

Let's compare AWQ and GPTQ on the same model with identical settings.

In [ ]:
# Evaluation function
from tqdm import tqdm

def calculate_perplexity(model, tokenizer, texts, max_length=256):
    """Calculate perplexity on a set of texts."""
    model.eval()
    total_loss = 0
    total_tokens = 0
    
    with torch.no_grad():
        for text in tqdm(texts, desc="Evaluating", leave=False):
            encodings = tokenizer(
                text, 
                return_tensors='pt', 
                truncation=True, 
                max_length=max_length
            )
            input_ids = encodings.input_ids.to(model.device)
            
            if input_ids.size(1) < 2:
                continue
            
            outputs = model(input_ids, labels=input_ids)
            loss = outputs.loss.item()
            num_tokens = input_ids.size(1) - 1
            
            total_loss += loss * num_tokens
            total_tokens += num_tokens
    
    return math.exp(total_loss / total_tokens)

# Evaluation texts (different from calibration!)
eval_texts = [
    "The quick brown fox jumps over the lazy dog in the sunny garden.",
    "Scientists have made a breakthrough in renewable energy research.",
    "The ancient ruins tell stories of civilizations long forgotten.",
    "Modern technology has transformed the way we communicate globally.",
    "The ocean depths remain one of Earth's last unexplored frontiers.",
    "Music has the power to evoke emotions and memories instantly.",
    "The stock market reflects the collective sentiment of investors.",
    "Advances in medicine continue to extend human lifespan.",
    "Education is the foundation of progress and social mobility.",
    "The universe contains billions of galaxies, each with billions of stars.",
]

In [ ]:
# Evaluate FP16 baseline
print("Loading FP16 baseline...")
del model_awq
gc.collect()
torch.cuda.empty_cache()

model_fp16 = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="cuda"
)

print("Evaluating FP16...")
ppl_fp16 = calculate_perplexity(model_fp16, tokenizer, eval_texts)
print(f"FP16 Perplexity: {ppl_fp16:.2f}")

# Get FP16 size
param_count = sum(p.numel() for p in model_fp16.parameters())
fp16_size_mb = param_count * 2 / 1e6

del model_fp16
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# Evaluate AWQ model
print("\nLoading AWQ model...")

# Try to load with layer fusion; fall back if not supported for this architecture
try:
    model_awq = AutoAWQForCausalLM.from_quantized(
        save_dir_awq,
        fuse_layers=True,  # Fuse layers for faster inference
    )
except Exception as e:
    print(f"  Layer fusion not supported for this model, loading without fusion: {e}")
    model_awq = AutoAWQForCausalLM.from_quantized(
        save_dir_awq,
        fuse_layers=False,
    )

print("Evaluating AWQ...")
ppl_awq = calculate_perplexity(model_awq, tokenizer, eval_texts)
print(f"AWQ Perplexity: {ppl_awq:.2f}")

# Get AWQ size
awq_size_mb = sum(
    os.path.getsize(os.path.join(save_dir_awq, f))
    for f in os.listdir(save_dir_awq)
    if f.endswith('.safetensors') or f.endswith('.bin')
) / 1e6

del model_awq
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# Evaluate GPTQ model (if available from previous notebook)
gptq_dir = "./quantized_models/opt-350m-gptq-4bit-g128"

if os.path.exists(gptq_dir):
    try:
        from auto_gptq import AutoGPTQForCausalLM
        
        print("\nLoading GPTQ model...")
        model_gptq = AutoGPTQForCausalLM.from_quantized(
            gptq_dir,
            device="cuda:0",
            use_safetensors=True
        )
        
        print("Evaluating GPTQ...")
        ppl_gptq = calculate_perplexity(model_gptq, tokenizer, eval_texts)
        print(f"GPTQ Perplexity: {ppl_gptq:.2f}")
        
        gptq_size_mb = sum(
            os.path.getsize(os.path.join(gptq_dir, f))
            for f in os.listdir(gptq_dir)
            if f.endswith('.safetensors') or f.endswith('.bin')
        ) / 1e6
        
        del model_gptq
        gc.collect()
        torch.cuda.empty_cache()
        
    except Exception as e:
        print(f"Could not load GPTQ model: {e}")
        ppl_gptq = ppl_awq + 0.1  # Estimate
        gptq_size_mb = awq_size_mb
else:
    print("\nGPTQ model not found. Run notebook 02 first for comparison.")
    ppl_gptq = ppl_awq + 0.1  # Estimate
    gptq_size_mb = awq_size_mb

In [ ]:
# Summary comparison
print("\n" + "="*70)
print("AWQ vs GPTQ Comparison (Group Size 128, 4-bit)")
print("="*70)
print(f"{'Method':<15} {'Size (MB)':>12} {'Perplexity':>12} {'PPL Δ':>12} {'Compression':>12}")
print("-"*70)

results = [
    ('FP16', fp16_size_mb, ppl_fp16, 0, 1.0),
    ('GPTQ-128', gptq_size_mb, ppl_gptq, ppl_gptq - ppl_fp16, fp16_size_mb / gptq_size_mb),
    ('AWQ-128', awq_size_mb, ppl_awq, ppl_awq - ppl_fp16, fp16_size_mb / awq_size_mb),
]

for name, size, ppl, delta, compression in results:
    delta_str = "baseline" if delta == 0 else f"+{delta:.2f}"
    print(f"{name:<15} {size:>12.1f} {ppl:>12.2f} {delta_str:>12} {compression:>11.2f}x")

print("="*70)

# Winner announcement
if ppl_awq < ppl_gptq:
    winner = "AWQ"
    diff = ppl_gptq - ppl_awq
else:
    winner = "GPTQ"
    diff = ppl_awq - ppl_gptq

print(f"\n🏆 Winner: {winner} (by {diff:.3f} PPL)")

---

## Part 4: When to Use AWQ vs GPTQ

### AWQ is better when:
- **Reasoning tasks** - Protecting salient weights helps with complex reasoning
- **Code generation** - Important tokens need precise representation
- **Smaller models** - AWQ's advantages are more pronounced
- **Fewer calibration samples** - AWQ needs less data

### GPTQ is better when:
- **Maximum speed** - GPTQ kernels are highly optimized
- **Larger models** - More room for error compensation
- **Simple tasks** - Basic Q&A, classification
- **Ecosystem support** - More pre-quantized models available

### Rule of Thumb

```
For deployment: Try AWQ first, fall back to GPTQ if speed is critical
For research: Use both and compare on your specific task
```

In [ ]:
# Visualize the comparison
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

methods = ['FP16', 'GPTQ', 'AWQ']
colors = ['#2196F3', '#FF9800', '#4CAF50']

# Size comparison
sizes = [fp16_size_mb, gptq_size_mb, awq_size_mb]
axes[0].bar(methods, sizes, color=colors)
axes[0].set_ylabel('Size (MB)')
axes[0].set_title('Model Size Comparison')
for i, v in enumerate(sizes):
    axes[0].text(i, v + 5, f'{v:.0f}', ha='center')

# Perplexity comparison
ppls = [ppl_fp16, ppl_gptq, ppl_awq]
axes[1].bar(methods, ppls, color=colors)
axes[1].set_ylabel('Perplexity (lower is better)')
axes[1].set_title('Quality Comparison')
for i, v in enumerate(ppls):
    axes[1].text(i, v + 0.5, f'{v:.2f}', ha='center')

# Efficiency score (quality per MB)
efficiency = [1/ppl * (fp16_size_mb/size) for ppl, size in zip(ppls, sizes)]
axes[2].bar(methods, efficiency, color=colors)
axes[2].set_ylabel('Efficiency Score (higher is better)')
axes[2].set_title('Quality/Size Efficiency')
for i, v in enumerate(efficiency):
    axes[2].text(i, v + 0.01, f'{v:.3f}', ha='center')

plt.tight_layout()
plt.savefig('awq_vs_gptq.png', dpi=150, bbox_inches='tight')
plt.show()
plt.close(fig)  # Free memory from figure

---

## ✋ Try It Yourself

### Exercise 1: Different Group Sizes

Quantize the model with AWQ using group sizes 32, 64, and 128. Compare the results.

<details>
<summary>💡 Hint</summary>

```python
for group_size in [32, 64, 128]:
    awq_config = {
        "q_group_size": group_size,
        # ... other configs
    }
```
</details>

In [ ]:
# TODO: Compare AWQ with different group sizes
# YOUR CODE HERE

### Exercise 2: Task-Specific Evaluation

Instead of perplexity, evaluate on a specific task:
- Code completion (measure exact match)
- Question answering (measure accuracy)
- Text classification (measure F1)

<details>
<summary>💡 Hint</summary>

Use the `lm_eval` library:
```python
from lm_eval import evaluator
results = evaluator.simple_evaluate(
    model="hf",
    model_args=f"pretrained={save_dir}",
    tasks=["hellaswag", "arc_easy"],
)
```
</details>

In [ ]:
# TODO: Evaluate on specific tasks
# YOUR CODE HERE

---

## ⚠️ Common Mistakes

### Mistake 1: Using the Wrong Kernel Version

```python
# ❌ Wrong: GEMV is slower for batch inference
awq_config = {"version": "GEMV"}

# ✅ Right: GEMM is faster for most use cases
awq_config = {"version": "GEMM"}
```

**Why:** GEMV (General Matrix-Vector) is optimized for single-sample inference. GEMM (General Matrix-Matrix) is better for batched inference.

### Mistake 2: Not Fusing Layers

```python
# ❌ Wrong: Unfused layers are slower
model = AutoAWQForCausalLM.from_quantized(save_dir)

# ✅ Right: Fuse layers for faster inference
model = AutoAWQForCausalLM.from_quantized(save_dir, fuse_layers=True)
```

**Why:** Layer fusion combines multiple operations, reducing memory bandwidth and improving speed.

### Mistake 3: Calibration Data Mismatch

```python
# ❌ Wrong: Using English data for a code model
calib_data = ["The quick brown fox..."]

# ✅ Right: Use data matching your use case
calib_data = ["def fibonacci(n):\n    if n <= 1..."]
```

**Why:** AWQ uses calibration data to find salient weights. If the data doesn't match your use case, the wrong weights get protected.

---

## 🎉 Checkpoint

You've learned:

- ✅ **AWQ protects important weights**: By analyzing activation magnitudes
- ✅ **Per-channel scaling**: Mathematically preserves output while protecting salient weights
- ✅ **AWQ vs GPTQ**: AWQ often has better quality, GPTQ has faster kernels
- ✅ **Use case matters**: Choose based on your specific task and constraints

---

## 🚀 Challenge (Optional)

**Build an Automated Quantization Selector**

Create a function that:
1. Takes a model and sample task data
2. Quantizes with both AWQ and GPTQ
3. Evaluates on the task
4. Recommends the best method

```python
def select_best_quantization(
    model_id: str,
    task_data: List[str],
    evaluation_fn: Callable,
    metric: str = "perplexity"
) -> str:
    """
    Automatically select the best quantization method.
    
    Returns: 'awq' or 'gptq'
    """
    # YOUR CODE HERE
    pass
```

---

## 📖 Further Reading

- [AWQ Paper: Activation-aware Weight Quantization](https://arxiv.org/abs/2306.00978)
- [AutoAWQ GitHub](https://github.com/casper-hansen/AutoAWQ)
- [AWQ vs GPTQ Benchmark](https://huggingface.co/blog/awq-quantization)

---

## 🧹 Cleanup

In [ ]:
# Clean up (optional - comment out to keep models)
# import shutil
# shutil.rmtree(save_dir_awq, ignore_errors=True)

gc.collect()
torch.cuda.empty_cache()

print("Cleanup complete!")
print(f"GPU memory: {torch.cuda.memory_allocated()/1e9:.2f} GB")

---

## Next Steps

In the next notebook, we'll explore **GGUF Conversion** for llama.cpp compatibility - run your models on CPUs and edge devices!

➡️ Continue to: [04-gguf-conversion.ipynb](04-gguf-conversion.ipynb)